<div>
<table style="width: 100%">
	<tr>
		<td>
		<table style="width: 100%">
			<tr>
                <td ><center><font size="5"><b>Module 49</b></font><center>
                <center><font size="6">Digital Innovations for Water Challenges</font><center></td>
			</tr>
			<tr>
                <td><center><font size="14">Notebook 1</font><center></td>
			</tr>
			<tr>
                <td><center><font size="6"><b>Acquire spatial data through OGC API's</b></font><center></td>
			</tr>
		</table>
		</td>
		<td><center><img src='images\ihe-delft-institute_unesco_fc-lr.jpg'></img></td>
	</tr>
</table>
</div>

# Table of contents
1. [Learning objectives](#learningobs)
2. [Introduction](#introduction)
3. [Accessing WCS layers](#access)
4. [Show WCS metadata](#metadata)
5. [Download a WCS layer](#download)
6. [Visualise the downloaded layer](#visualise)

# 1. Learning objectives<a name="learningobs"></a>

- To define different OGC API's
- To access WCS layers
- To show WCS metadata
- To download WCS layers
- To visualise the downloaded layer

# 2. Introduction<a name="introduction"></a>

Many geospatial data are accessible through standards and API's from the [Open Geospatial Consortium](https://www.ogc.org/) (OGC). OGC is an international non-profit foundation, more than 500 commercial, governmental, nonprofit and research organizations collaborate in a consensus process encouraging development and implementation of open standards for geospatial content and services, sensor web and Internet of Things, GIS data processing and data sharing.

The most important standards are:

| **Standard** |  **Description** | **Data model** |
|:------------:|:----------------:|:--------------:|
| WMS          | Web Map Service  | Rendered picture of data |
| WFS          | Web Feature Service | Vector data |
| WCS          | Web Coverage Service | Raster data |

Watch [this video](https://youtu.be/XjjqQIXumvA) for more information.

We're going to use the [OWSLib](https://geopython.github.io/OWSLib/) package to connect to these services and we're going to use the [Rasterio](https://rasterio.readthedocs.io/en/latest/) package to save WCS layer to a GeoTIFF. Make sure that these packages are installed in your conda environment.

We'll demonstrate this for data from [FAO WaPOR](https://wapor.apps.fao.org).

# 3. Accessing WCS layer<a name="access"></a>
In this example we're going to connect to the WCS layers available in FAO WaPOR.

Let's first check which layers are available.

In [ ]:
from owslib.wcs import WebCoverageService

# Access the WCS by proving the url and optional arguments (here version)
wcs = WebCoverageService('https://io.apps.fao.org/geoserver/wcs?', version='1.0.0')

Now we have connection with the WCS, we can print the layers. This is better readable if we convert it to a list.

In [ ]:
# Print to check the contents of the WCS
layers = list(wcs.contents)
print(layers)

Some explanation about the layer naming, which you could also figure out by exploring the WaPOR portal:
WAPOR_2: means WaPOR version 2
l1, l2 or l3 give the level, respectively 250, 100 and 30 m data.
Then for l3 you see 3 characters describing the study area, e.g. awa is Awash. This is followed by the data description, e.g. aeti.
The last character shows the temporal resolution: a is annual, m is monthly and d is dekadal (10 days).

In this example we're looking for WaPOR version 2, level 3 data. We can find those layers by looking for the substring `WAPOR_2:l3_`. We can do this with a simple `for` loop.

In [ ]:
substring = 'WAPOR_2:l3_'
for text in layers:
    if substring in text:
        print(text)

Now use similar code to find the WaPOR Level 3 AETI layer at a monthly resolution for Zankalon (Egypt).

# 4. Show WCS metadata <a name="metadata"></a>
We can list all operations that are available with the following code:

In [ ]:
# Get all operations and print the name of each of them
print([op.name for op in wcs.operations])

These are standard OGC protocols. It allows access to the data (GetCoverage), the metadata (DescribeCoverage), and the capabilities (GetCapabilities).
Let's have a look at the metadata for `WAPOR_2:l3_zan_aeti_m`.

In [ ]:
metadata = wcs.contents['WAPOR_2:l3_zan_aeti_m']

# print supported coordinate reference systems
print(metadata.supportedCRS)

# print the bounding box in WGS 84 coordinates
print(metadata.boundingBoxWGS84)

# print the supported file formats
print(metadata.supportedFormats)

Now try this for `WAPOR_2:l3_kog_t_d`

# 5. Download a WCS layer <a name="download"></a>
Let's now store the data locally. We'll use `wcs.getCoverage`. It needs:
* `identifier`: layer name that we've found through `wcs.contents` before
* `bbox`: bounding box in the supported reference system
* `format`: one of the supported file formats
* `crs`: the supported coordinate reference system
* `xres`: resolution in x-dimension in units of the crs
* `yres`: resolution in y-dimension in units of the crs

Below we'll download the `WAPOR_2:l3_zan_aeti_m` layer and save it in the `data` subfolder as `l3_zan_aeti_m.tif`:

In [ ]:
import os

bbox = (317985,3348885,377685,3411885)

if not os.path.exists('data'):
    os.makedirs('data')
    
response = wcs.getCoverage(identifier = 'WAPOR_2:l3_zan_aeti_m', bbox=bbox, format='GeoTIFF', crs = 'urn:ogc:def:crs:EPSG::32636',resx=30,resy=30)

with open('./data/l3_zan_aeti_m.tif', 'wb') as file:
    file.write(response.read())

# 6. Visualise the downloaded layer <a name="visualise"></a>
Now the raster has been downloaded, let's have a look at it.
We'll use the *rasterio* package for that.

In [ ]:
import rasterio
from rasterio.plot import show

# open the raster layer
aeti = rasterio.open('./data/l3_zan_aeti_m.tif', driver='GTiff')

# print the metadata
print(aeti.meta)

# plot the map
show(aeti, title='AETI Zankalon', cmap='nipy_spectral')

Now repeat the downloading and visualisation for another WCS layer.

For vector data you can use `owslib.wfs`. See [this page](https://geopython.github.io/OWSLib/usage.html#wfs) for examples.